In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.components.memory import MemoryArray
from sequence.topology.node import Node
from sequence.resource_management.resource_manager import ResourceManager

# cria uma instância de Timeline
runtime = 1e12  # tempo máximo de simulação em picosegundos
tl = Timeline(runtime)

# Hub tem que ter memória, gerenciar os entrelaçamentos
class Hub(Node):
    """Nó responsável por gerenciar a memória e os entrelaçamentos. Fica entre os sensores e o gateway de saída."""
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        # Memória de qubits, são esses argumentos
        memory_array_name = f"{self.name}.memory_array"
        # outros detalhes envolvem parametros de configuração da memória, mas por enquanto só isso
        self.memory_array = MemoryArray(memory_array_name, timeline, num_memories=10)
        # adiciona as memórias ao hub
        self.add_component(self.memory_array)
        # indica que a memória é desse hub
        self.memory_array.add_receiver(self)
        # gerenciar os entrelaçamentos (pode ser melhor definir o próprio, mas por enquanto é assim)
        self.resource_manager = ResourceManager(self, memory_array_name)
    
    def get(self, photon, **kwargs):
        """
        Aqui fica toda a lógica de como o hub vai lidar com os qubits que chegam.
        Por exemplo, podemos descartar se vier de algum nó; criar pares EPR ou descartar memórias antigas, etc
        """
        src = kwargs.get("src")
        # percorre a lista de memórias (pelo memory manager mesmo) do hub e verifica se alguma está livre (RAW)
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                print(f"[Hub {self.name}] recebeu qubit {photon.name} de {src.name}.")
                info.memory.add_photon(photon) # isso é opcional pq é meio que de mentirinha, o quantummanager faz o trabalho de verdade
                self.resource_manager.update(None, info.memory, "ENTANGLE")
    
        
hub = Hub("hub", tl)
for m in hub.resource_manager.memory_manager:
    print(type(m))
    print(f"[Hub {hub.name}] memória {m.memory.name} está {m.state}.")
# configuração opcional: desativar barra de progresso
tl.show_progress = False

<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[0] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[1] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[2] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[3] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[4] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[5] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[6] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[7] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_ar